# PURPOSE

In [ ]:
'''
use bottom-up approach
'''
None

# IMPORTS

In [ ]:
%run ipynb_setup.ipynb

In [ ]:
%run class_def.ipynb

# GET DATA

In [ ]:
d=Dataset()
df_num = d.df.select_dtypes(include=['int64','float64'])
df_num

# REPEAT WITH PIPELINE

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import GridSearchCV

In [ ]:
import pandas as pd
import numpy as np
import scipy.cluster.hierarchy as sch

### [figure out best `k` to use] Impute + StandardScaler (+ MinMaxScaler)

In [ ]:
# pipeline
pl = make_pipeline(
    SimpleImputer(missing_values=np.nan, strategy='mean'), # PCA can't have nans
    StandardScaler(), # apply standard scaler first, PCA super sensitive to scaling
    #MinMaxScaler(feature_range=(-1,1)), # consider apply minmax scaler on 1 sd of the data as well smooth outliers
)
reduced = pl.fit_transform(df_num)

In [ ]:
ac=AgglomerativeClustering(memory='mycachedir',compute_full_tree=True)

In [ ]:
ac.fit(reduced)

In [ ]:
ac

In [ ]:
pd.Series(ac.labels_).unique()

In [ ]:
ac.plot()

In [ ]:
classifier = GridSearchCV(
    ac,
    {n_clusters : range(2,6)},
    scoring = 'adjusted_rand_score',
    n_jobs=-1,
    verbose=2,
)
classifier.fit(reduced)

In [ ]:
model = AgglomerativeClustering(n_clusters=5, affinity='euclidean', linkage='ward')
model.fit(reduced)
labels = model.labels_

In [ ]:
pd.Series(labels).value_counts()

### [figure out best `k` to use] Impute + StandardScaler + MinMaxScaler + PCA

In [ ]:
# pipeline
pl = make_pipeline(
    SimpleImputer(missing_values=np.nan, strategy='mean'), # PCA can't have nans
    StandardScaler(), # apply standard scaler first, PCA super sensitive to scaling
    MinMaxScaler(feature_range=(-1,1)), # consider apply minmax scaler on 1 sd of the data as well smooth outliers
    PCA(n_components=5),
)

reduced = pl.fit_transform(df_num)

# A list holds the SSE values for each k
sse     = {}
k_range = range(1, 50) 
for k in k_range:
    kmeans = KMeans(
        init         = "random",
        n_clusters   = k, 
        n_init       = 10,
        max_iter     = 300,
        random_state = 42
    )
    kmeans.fit(reduced)
    sse[k] = kmeans.inertia_

pd.Series(sse).plot()